<a href="https://colab.research.google.com/github/insooAI/TNT_2021_winter_team_1/blob/main/4%EC%A3%BC%EC%B0%A8_%EA%B9%80%EC%9D%B8%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import numpy as np


In [ ]:
!pip install pycaret

In [5]:
from pycaret.utils import version
version()

'2.2.3'

# EDA

## load data

In [75]:
tr = pd.read_csv('/content/drive/My Drive/데이터/4주차/bicycle/train.csv')
te = pd.read_csv('/content/drive/My Drive/데이터/4주차/bicycle/test.csv')
sub = pd.read_csv('/content/drive/My Drive/데이터/4주차/bicycle/submission.csv')

In [13]:
display(tr,te)

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0
...,...,...,...,...,...,...,...,...,...,...,...
1454,2174,4,16.8,0.0,1.6,53.0,2000.0,0.031,37.0,27.0,21.0
1455,2175,3,10.8,0.0,3.8,45.0,2000.0,0.039,34.0,19.0,20.0
1456,2176,5,18.3,0.0,1.9,54.0,2000.0,0.009,30.0,21.0,22.0
1457,2178,21,20.7,0.0,3.7,37.0,1395.0,0.082,71.0,36.0,216.0


,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
0,0,7,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0
1,1,17,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0
2,2,13,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0
3,4,6,22.5,0.0,2.5,60.0,1185.0,0.027,52.0,38.0
4,5,22,14.6,1.0,3.4,93.0,218.0,0.041,18.0,15.0
...,...,...,...,...,...,...,...,...,...,...
710,2148,1,24.6,0.0,2.4,60.0,1745.0,NaN,NaN,NaN
711,2149,1,18.1,0.0,1.0,55.0,2000.0,NaN,NaN,NaN
712,2165,9,23.3,0.0,2.3,66.0,1789.0,0.020,17.0,15.0
713,2166,16,27.0,0.0,1.6,46.0,1956.0,0.032,40.0,26.0


In [18]:
def info(alldata):
    colname = []
    dtype = []
    unique_value = []
    unique_count = []
    for col in alldata.columns[:-1]:
        colname.append(col)
        dtype.append(alldata[col].dtype)
        unique_value.append(alldata[col].unique())
        unique_count.append(alldata[col].nunique())
    
    info = pd.DataFrame({'colname': colname, 'dtype' : dtype, 'unique_value':unique_value, 'unique_count':unique_count})
    # df = pd.DataFrame([colname, dtype, unique_value, unique_count], index = ['colname','dtype','unique_value','unique_count']).transpose()
    # df # 리스트로 데프 만들기 # 가로로 들어가기때문에 transpose 해줘야
    return info

In [19]:
info(tr)

,colname,dtype,unique_value,unique_count
0,id,int64,"[3, 6, 7, 8, 9, 13, 14, 16, 19, 20, 21, 22, 24...",1459
1,hour,int64,"[20, 13, 6, 23, 18, 2, 3, 21, 9, 14, 4, 10, 1,...",24
2,hour_bef_temperature,float64,"[16.3, 20.1, 13.9, 8.1, 29.5, 13.6, 10.6, 16.0...",245
3,hour_bef_precipitation,float64,"[1.0, 0.0, nan]",2
4,hour_bef_windspeed,float64,"[1.5, 1.4, 0.7, 2.7, 4.8, 1.7, 6.0, 1.9, 2.1, ...",73
5,hour_bef_humidity,float64,"[89.0, 48.0, 79.0, 54.0, 7.0, 80.0, 58.0, 21.0...",91
6,hour_bef_visibility,float64,"[576.0, 916.0, 1382.0, 946.0, 2000.0, 1073.0, ...",781
7,hour_bef_ozone,float64,"[0.027000000000000003, 0.042, 0.033, 0.04, 0.0...",95
8,hour_bef_pm10,float64,"[76.0, 73.0, 32.0, 75.0, 27.0, 34.0, 62.0, 90....",148
9,hour_bef_pm2.5,float64,"[33.0, 40.0, 19.0, 64.0, 11.0, 15.0, 28.0, 18....",79


In [21]:
tr.isnull().sum()

id                          0
hour                        0
hour_bef_temperature        2
hour_bef_precipitation      2
hour_bef_windspeed          9
hour_bef_humidity           2
hour_bef_visibility         2
hour_bef_ozone             76
hour_bef_pm10              90
hour_bef_pm2.5            117
count                       0
dtype: int64

In [76]:
tr = tr.drop('id',1)
te = te.drop('id',1)

# Modeling


## 저절로 numeric인지 categorical인지 판단해주는 pycaret
- transformed train set : train data를 train set으로 분리 + 전처리
- transformed test set: train data를 test(valid)set으로 분리 + 전처리
** 여기서는 1459개의 row가 1021개 + 438개로 분리되었고 column이 10개에서 11개로 확장 
-...	hour_bef_precipitation (원 핫 인코딩)
- setup : default로 30% : train set, 70% : test set으로 분리해놓고, 그 30%는 holdout 함 (나중에 predict에 이용) 
** 즉, predict_model(automl 모델) : 70퍼만 학습한 모델 
** finalize_model로 비로소 전체 train에 학습 가능  

In [55]:
from pycaret.regression import *
reg1 = setup(tr, target = 'count', session_id=123, log_experiment=True, experiment_name='bicycle')

,Description,Value
0,session_id,123
1,Target,count
2,Original Data,"(1459, 10)"
3,Missing Values,True
4,Numeric Features,8
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1021, 11)"


In [25]:
best_model = compare_models(fold=5, n_select=5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,27.2049,1.555849e+03,39.3319,7.788000e-01,0.5406,0.7739,1.418
et,Extra Trees Regressor,27.1410,1.564056e+03,39.4497,7.769000e-01,0.5065,0.7345,0.558
gbr,Gradient Boosting Regressor,28.2285,1.654410e+03,40.5248,7.648000e-01,0.5323,0.7984,0.132
lightgbm,Light Gradient Boosting Machine,28.7836,1.764483e+03,41.8741,7.480000e-01,0.5637,0.8310,0.078
rf,Random Forest Regressor,29.3290,1.794218e+03,42.3057,7.433000e-01,0.5281,0.7985,0.680
xgboost,Extreme Gradient Boosting,29.0221,1.864814e+03,43.1149,7.323000e-01,0.5618,0.7258,4.542
ada,AdaBoost Regressor,43.4302,2.825499e+03,53.1346,5.913000e-01,0.8145,1.6568,0.116
lr,Linear Regression,40.8198,2.927260e+03,54.0430,5.816000e-01,0.7680,1.1824,0.506
ridge,Ridge Regression,40.9579,2.947680e+03,54.2371,5.787000e-01,0.7570,1.1757,0.016
lasso,Lasso Regression,41.1072,2.985579e+03,54.5734,5.732000e-01,0.7827,1.2740,0.016


In [39]:
# blended가 단일 catboost보다 떨어지는 듯
blended = blend_models(estimator_list = best_model, fold = 5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,28.4770,1725.3759,41.5376,0.7525,0.5113,0.6568
1,25.3405,1403.5236,37.4636,0.7879,0.5672,1.2440
2,28.5076,1904.9723,43.6460,0.7444,0.4511,0.4679
3,27.9438,1548.2273,39.3475,0.8131,0.4049,0.4495
4,24.7003,1260.7293,35.5067,0.7844,0.5920,0.8982
Mean,26.9938,1568.5657,39.5003,0.7765,0.5053,0.7433
SD,1.6363,228.1117,2.8795,0.0250,0.0699,0.2981


## 0. Creat catboost Model for differntiating learning rates

---



In [35]:
cat = create_model('catboost')
cats = [create_model('catboost', learning_rate = i) for i in np.arange(0.01,0.1,0.01)]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,27.1935,1493.8337,38.6501,0.8112,0.5048,0.4632
1,27.4211,1541.0555,39.2563,0.7401,0.5513,0.7201
2,26.1338,1544.0709,39.2947,0.7747,0.7441,2.5267
3,25.8921,1573.0283,39.6614,0.7524,0.4689,0.3708
4,29.0952,2133.0562,46.1850,0.7486,0.5406,0.7062
5,25.7508,1246.4769,35.3055,0.8059,0.4067,0.3464
6,28.7011,1527.0663,39.0777,0.8328,0.4963,0.3949
7,28.8032,1686.3651,41.0654,0.7717,0.5029,0.4483
8,23.5605,1093.4351,33.0671,0.8076,0.6877,1.1952
9,23.3175,1162.2386,34.0916,0.8060,0.4749,0.4406


## 1. Tuning catboost Model

In [37]:
tuned_cat = tune_model(cat, n_iter = 50, optimize = 'RMSE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,28.2377,1697.5211,41.2010,0.7854,0.4682,0.4980
1,30.1517,1797.1226,42.3925,0.6969,0.6133,0.9762
2,26.0747,1424.7877,37.7464,0.7921,0.7086,2.3442
3,26.2536,1422.0386,37.7099,0.7762,0.4682,0.4768
4,29.3319,2223.1506,47.1503,0.7380,0.5288,0.6677
5,26.5857,1217.8997,34.8984,0.8103,0.4290,0.4380
6,29.0373,1513.1061,38.8987,0.8343,0.4141,0.4059
7,30.4791,1855.7664,43.0786,0.7488,0.4812,0.6947
8,26.3138,1323.3591,36.3780,0.7671,0.7923,1.9907
9,26.3341,1391.5093,37.3029,0.7677,0.4997,0.5492


## 2. Ensemble Model
- 평균 RMSE ... bagging 이 boosting보다 better

In [56]:
cat = create_model('catboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,25.6578,1455.9189,38.1565,0.8160,0.4226,0.4165
1,27.7294,1477.3428,38.4362,0.7509,0.5522,0.8173
2,24.0112,1306.2346,36.1419,0.8094,0.7403,2.3411
3,25.2014,1398.8055,37.4006,0.7798,0.4465,0.3896
4,29.5383,2361.7690,48.5980,0.7217,0.5413,0.7162
5,25.2571,1168.3730,34.1815,0.8180,0.3702,0.3571
6,28.2460,1399.0000,37.4032,0.8468,0.4489,0.3422
7,26.8172,1562.2616,39.5255,0.7885,0.4793,0.4770
8,22.3692,997.7788,31.5876,0.8244,0.6359,0.9865
9,22.7055,1068.2054,32.6834,0.8217,0.4637,0.4393


In [57]:
# method default : bagging
bagged_cat = ensemble_model(cat, n_estimators=50)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,26.1647,1505.0384,38.7948,0.8098,0.4329,0.4406
1,27.8070,1517.1244,38.9503,0.7442,0.5736,0.8678
2,24.6230,1365.7042,36.9554,0.8007,0.6974,2.2107
3,24.6984,1304.8162,36.1222,0.7946,0.3990,0.3695
4,28.4595,2108.9104,45.9229,0.7515,0.5322,0.6975
5,25.5948,1173.7865,34.2606,0.8172,0.3848,0.3583
6,27.9972,1376.8569,37.1060,0.8493,0.3896,0.3635
7,27.5778,1655.4111,40.6867,0.7759,0.4430,0.5255
8,24.4022,1152.8645,33.9539,0.7971,0.7107,1.3886
9,23.6650,1170.1821,34.2079,0.8046,0.4570,0.4723


In [66]:
boost_cat = ensemble_model(cat, method = 'Boosting')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,27.7360,1583.7556,39.7964,0.7998,0.4562,0.4854
1,28.4668,1522.9090,39.0245,0.7432,0.6107,0.9789
2,24.4893,1277.2590,35.7388,0.8136,0.7079,2.2256
3,24.6121,1239.0542,35.2002,0.8050,0.4141,0.3900
4,28.8704,2091.5859,45.7339,0.7535,0.5539,0.7192
5,25.8714,1167.6077,34.1703,0.8181,0.3775,0.3687
6,28.8662,1484.2064,38.5254,0.8375,0.4327,0.3876
7,27.3963,1636.3745,40.4521,0.7785,0.4236,0.4125
8,24.7737,1184.2055,34.4123,0.7916,0.6850,1.2030
9,23.4926,1118.7207,33.4473,0.8132,0.4865,0.5069


In [68]:
pred_holdout = predict_model(boost_cat)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,25.468,1333.9742,36.5236,0.779,0.5478,0.7573


### 현재까지의 실험

---
- train 데이터를 다시한번 train set과 validation set으로 나눠서실험을 한 것으로, 전체 train 데이터에 대한 학습이 아님
- finalzie_model 해야 전체 train 데이터에 대해 학습하는 것


In [59]:
final_model = finalize_model(bagged_cat)

In [60]:
predictions = predict_model(final_model, data = te)

In [61]:
predictions

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,Label
0,7,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0,112.835136
1,17,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0,206.500827
2,13,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0,51.658120
3,6,22.5,0.0,2.5,60.0,1185.0,0.027,52.0,38.0,56.860576
4,22,14.6,1.0,3.4,93.0,218.0,0.041,18.0,15.0,49.116443
...,...,...,...,...,...,...,...,...,...,...
710,1,24.6,0.0,2.4,60.0,1745.0,NaN,NaN,NaN,70.006542
711,1,18.1,0.0,1.0,55.0,2000.0,NaN,NaN,NaN,63.818779
712,9,23.3,0.0,2.3,66.0,1789.0,0.020,17.0,15.0,112.533821
713,16,27.0,0.0,1.6,46.0,1956.0,0.032,40.0,26.0,157.480681


In [62]:
sub['count'] = predictions['Label']
sub

,id,count
0,0,112.835136
1,1,206.500827
2,2,51.658120
3,4,56.860576
4,5,49.116443
...,...,...
710,2148,70.006542
711,2149,63.818779
712,2165,112.533821
713,2166,157.480681


In [65]:
sub.to_csv('submission1.csv', index=False)

## 순위 올리기
- 너무 과적합되어있는거 같아 bagged_cat에서 다시 (n_estimator 조정 : 별 차이없음)
- catboost를 bagging하는 전략이었는데, 차라리 다른 모델들과 blending하는게 나을지도?

In [69]:
bagged_cat = ensemble_model(cat)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,26.3524,1513.1718,38.8995,0.8087,0.4449,0.4624
1,27.4871,1471.1118,38.3551,0.7519,0.5666,0.8463
2,24.5877,1354.7649,36.8071,0.8023,0.6946,2.1735
3,24.6760,1327.8888,36.4402,0.7910,0.4177,0.3934
4,28.9422,2162.9857,46.5079,0.7451,0.5363,0.7056
5,25.8570,1210.6281,34.7941,0.8114,0.4001,0.3591
6,27.2098,1276.2920,35.7252,0.8603,0.3947,0.3539
7,27.4763,1616.5208,40.2060,0.7812,0.4463,0.5354
8,24.0676,1129.2187,33.6038,0.8013,0.7040,1.3366
9,23.7397,1215.6091,34.8656,0.7970,0.4630,0.4812


In [70]:
pred_holdout = predict_model(bagged_cat)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,24.8294,1308.4836,36.173,0.7833,0.528,0.7582


In [71]:
final_model = finalize_model(bagged_cat)

In [72]:
predictions = predict_model(final_model, data = te)
sub['count'] = predictions['Label']
sub.to_csv('submission1.csv', index=False)

## blending

In [128]:
tr = pd.read_csv('/content/drive/My Drive/데이터/4주차/bicycle/train.csv')
te = pd.read_csv('/content/drive/My Drive/데이터/4주차/bicycle/test.csv')
sub = pd.read_csv('/content/drive/My Drive/데이터/4주차/bicycle/submission.csv')

tr = tr.drop('id',1)
te = te.drop('id',1)

In [140]:
best_3 = compare_models(sort = 'RMSE', n_select=5)
 # 모델이 저장되는 것 
 # 모델 순위가 바뀌지 않는 한, n_select는 학습에는 영향없음 ... 완전 똑같음

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,24.5382,1537.2723,36.9005,0.6786,0.5003,0.8070,0.066
catboost,CatBoost Regressor,26.0975,1473.3131,38.1327,0.7825,0.5281,0.8233,1.217
et,Extra Trees Regressor,26.2321,1578.7926,39.3578,0.7677,0.5012,0.8150,0.501
rf,Random Forest Regressor,26.9926,1689.4038,40.8276,0.7515,0.5024,0.8270,0.634
gbr,Gradient Boosting Regressor,28.0430,1732.1392,41.3061,0.7453,0.5277,0.8719,0.114
xgboost,Extreme Gradient Boosting,27.1572,1745.7428,41.4159,0.7425,0.5281,0.8423,4.560
ada,AdaBoost Regressor,42.7689,2736.3524,52.2307,0.5940,0.7990,1.6608,0.110
dt,Decision Tree Regressor,33.3125,2796.6620,52.6229,0.5850,0.6106,0.9171,0.020
lr,Linear Regression,40.2968,2854.8844,53.0369,0.5831,0.7688,1.2302,0.274
ridge,Ridge Regression,40.3816,2870.5545,53.1993,0.5805,0.7674,1.2303,0.015


In [138]:
# plot_model(estimator = blended, plot = 'feature')

## 코드 합치기

In [147]:
reg = setup(tr, target = 'count', session_id=0)
best_5 = compare_models(sort = 'RMSE', n_select=5)
blend = blend_models(estimator_list = best_5, fold = 5)
pred_holdout = predict_model(blend)
final_model = finalize_model(blend)
predictions = predict_model(final_model, data = te)
predictions

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,28.4010,1829.7102,42.7751,0.7521,0.5235,0.6643
1,22.0507,987.8973,31.4308,0.8371,0.4888,0.7787
2,27.7235,1592.2401,39.9029,0.7764,0.5001,0.7147
3,25.3777,1468.7431,38.3242,0.7998,0.5562,1.2991
4,24.1311,1351.5846,36.7639,0.7930,0.5494,0.8358
Mean,25.5368,1446.0351,37.8394,0.7917,0.5236,0.8585
SD,2.3298,278.5352,3.7704,0.0280,0.0264,0.2278


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,23.3922,1218.1015,34.9013,0.8161,0.4333,0.4282


,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,Label
0,7,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0,109.388169
1,17,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0,201.916677
2,13,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0,49.902736
3,6,22.5,0.0,2.5,60.0,1185.0,0.027,52.0,38.0,45.066150
4,22,14.6,1.0,3.4,93.0,218.0,0.041,18.0,15.0,46.901717
...,...,...,...,...,...,...,...,...,...,...
710,1,24.6,0.0,2.4,60.0,1745.0,NaN,NaN,NaN,67.871651
711,1,18.1,0.0,1.0,55.0,2000.0,NaN,NaN,NaN,67.765500
712,9,23.3,0.0,2.3,66.0,1789.0,0.020,17.0,15.0,128.848744
713,16,27.0,0.0,1.6,46.0,1956.0,0.032,40.0,26.0,151.130876


In [148]:
reg = setup(tr, target = 'count', session_id=2)
best_5 = compare_models(sort = 'RMSE', n_select=5)
blend = blend_models(estimator_list = best_5, fold = 5)
pred_holdout = predict_model(blend)
final_model = finalize_model(blend)
predictions2 = predict_model(final_model, data = te)
predictions2

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,-0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
1,26.5371,1502.1246,38.7573,0.7961,0.4200,0.4135
2,26.9208,1732.8013,41.6269,0.7112,0.5627,0.8483
3,24.9342,1482.1306,38.4984,0.8006,0.5900,1.6055
4,26.3397,1446.7009,38.0355,0.7683,0.5149,0.7940
Mean,20.9464,1232.7515,31.3836,0.6152,0.4175,0.7323
SD,10.4947,624.5402,15.7423,0.3093,0.2166,0.5325


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,25.3571,1300.946,36.0686,0.8063,0.5592,0.8378


,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,Label
0,7,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0,100.475394
1,17,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0,225.814505
2,13,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0,52.482055
3,6,22.5,0.0,2.5,60.0,1185.0,0.027,52.0,38.0,46.147450
4,22,14.6,1.0,3.4,93.0,218.0,0.041,18.0,15.0,59.321085
...,...,...,...,...,...,...,...,...,...,...
710,1,24.6,0.0,2.4,60.0,1745.0,NaN,NaN,NaN,67.060114
711,1,18.1,0.0,1.0,55.0,2000.0,NaN,NaN,NaN,68.198715
712,9,23.3,0.0,2.3,66.0,1789.0,0.020,17.0,15.0,118.957694
713,16,27.0,0.0,1.6,46.0,1956.0,0.032,40.0,26.0,152.669432


In [172]:
final1 = predictions * 0.81 + predictions2 *0.19

In [173]:
sub['count'] = final1.Label
sub

,id,count
0,0,107.694742
1,1,206.457265
2,2,50.392807
3,4,45.271597
4,5,49.261397
...,...,...
710,2148,67.717459
711,2149,67.847811
712,2165,126.969444
713,2166,151.423202


In [174]:
sub.to_csv('f1.csv',index=False)
# 44.62756 

In [176]:
reg = setup(tr, target = 'count', session_id=10)
best_3 = compare_models(sort = 'RMSE', n_select=3)
blend = blend_models(estimator_list = best_3, fold = 3)
pred_holdout = predict_model(blend)
final_model = finalize_model(blend)
predictions3 = predict_model(final_model, data = te)
predictions3

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,23.6648,1165.1507,34.1343,0.8493,0.3801,0.3302
1,29.3859,1738.3757,41.6938,0.7548,0.6482,1.6369
2,23.9987,1317.3296,36.2950,0.7880,0.4460,0.4359
Mean,25.6831,1406.9520,37.3744,0.7974,0.4914,0.8010
SD,2.6218,242.4470,3.1791,0.0392,0.1140,0.5926


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,25.553,1415.476,37.6228,0.7762,0.5599,0.8557


,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,Label
0,7,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0,111.356372
1,17,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0,217.526538
2,13,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0,46.027803
3,6,22.5,0.0,2.5,60.0,1185.0,0.027,52.0,38.0,52.099975
4,22,14.6,1.0,3.4,93.0,218.0,0.041,18.0,15.0,49.890480
...,...,...,...,...,...,...,...,...,...,...
710,1,24.6,0.0,2.4,60.0,1745.0,NaN,NaN,NaN,70.494595
711,1,18.1,0.0,1.0,55.0,2000.0,NaN,NaN,NaN,65.974212
712,9,23.3,0.0,2.3,66.0,1789.0,0.020,17.0,15.0,122.834871
713,16,27.0,0.0,1.6,46.0,1956.0,0.032,40.0,26.0,147.412037


In [184]:
final2 = predictions * 0.8 + predictions3 * 0.1 + predictions2 * 0.1
final2

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,Label
0,7.0,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0,108.693712
1,17.0,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0,205.867446
2,13.0,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0,49.773175
3,6.0,22.5,0.0,2.5,60.0,1185.0,0.027,52.0,38.0,45.877662
4,22.0,14.6,1.0,3.4,93.0,218.0,0.041,18.0,15.0,48.442530
...,...,...,...,...,...,...,...,...,...,...
710,1.0,24.6,0.0,2.4,60.0,1745.0,NaN,NaN,NaN,68.052792
711,1.0,18.1,0.0,1.0,55.0,2000.0,NaN,NaN,NaN,67.629693
712,9.0,23.3,0.0,2.3,66.0,1789.0,0.020,17.0,15.0,127.258252
713,16.0,27.0,0.0,1.6,46.0,1956.0,0.032,40.0,26.0,150.912848


In [185]:
sub['count'] = final2.Label
sub.to_csv('f2.csv',index=False)

## 32위 ( 점수 : 44.626916026 ) 로 마무리
- 최종 모델 : blended model 3개를 앙상블 (0.8 / 0.1 / 0.1)